## Install Required Supporting Libraries

In [ ]:
%pip install --q semantic-link

## BQ Sync Python Package
If you are not going to leverage an environment, the BQ Sync package needs to be installed at runtime. 

<strong>Options for Loading/Using the BQ Sync Package</strong>
1. Runtime from OneLake (stable): <br />
    <code>%pip install /lakehouse/default/Files/BQ_Sync_Process/libs/FabricSync-0.1.0-py3-none-any.whl</code>
2. Runtime from GitHub (latest version): <br/>
    <code>%pip install https://github.com/microsoft/FabricBQSync/raw/main/Packages/FabricSync/dist/FabricSync-0.1.0-py3-none-any.whl</code>
3. From Spark Environment 

<strong>Please note that if you are scheduling this notebook to run from a pipeline, you must provide the <code>_inlineInstallationEnabled</code> parameter to the pipeline for pip install support.</strong>

In [ ]:
%pip install --upgrade --force-reinstall https://github.com/microsoft/FabricBQSync/raw/main/Packages/FabricSync/dist/FabricSync-0.1.0-py3-none-any.whl

In [ ]:
from FabricSync.FabricAPI import *
from FabricSync.DeltaStorageInventory import *

## Configuration for the OneLake Delta Inventory

In [ ]:
lakehouse_name = "DW" #The Lakehouse you want to inventory

metadata_lakehouse = "Sync_Metadata" #The Lakehouse were the inventory output will be stored
parallelism = 5 #Degree of parallelism to use for the lakehouse inventory process
track_history = False #Track inventory history overtime (you must account for this history when running queries)
async_process = True #Process the lakehouse inventory asynchronously in parallel

In [ ]:
fabric_api = FabricAPIUtil()
workspace_id = fabric_api.get_workspace_id()
lakehouse_id = fabric_api.get_lakehouse_id(workspace_id, lakehouse_name)

fabric_api.get_or_create_lakehouse(workspace_id, metadata_lakehouse)

delta_inventory = DeltaStorageInventory(session= spark, \
    target_lakehouse=metadata_lakehouse, 
    parallelism=parallelism,
    track_history=track_history,
    async_process=async_process)

delta_inventory.run_onelake_lakehouse_catalog(workspace_id=workspace_id, \
    lakehouse_id=lakehouse_id, \
    lakehouse_name=lakehouse_name)